In [ ]:
import os
import pickle

from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import units as u
import astropy_healpix as ah
import ligo.skymap.plot
import ligo.skymap.io
import ligo.skymap.postprocess
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from thesis_utils.plotting import save_figure
import thesis_utils.colours as thesis_colours

# set_plotting()
# Breaks formatting for skymaps, so set manually
sns.set_palette("colorblind")
plt.rcParams["figure.figsize"] = (6.141732, 3.79579912579)
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [ ]:
import ligo.skymap

ligo.skymap.__version__

In [ ]:
obs_time = Time(1126259642.413, format="gps")

injected = dict(
    ra=1.375,
    dec=0.2108,
)

et_location = dict(
    latitude=43 + 37.0 / 60 + 53.0921 / 3600,
    longitude=10 + 30.0 / 60 + 16.1878 / 3600,
)


ce_location = dict(
    latitude=46 + 27.0 / 60 + 18.528 / 3600,
    longitude=-(119 + 24.0 / 60 + 27.5657 / 3600),
)

In [ ]:
gmst = obs_time.sidereal_time("mean", "greenwich").rad

In [ ]:
et_ra = et_location["longitude"] * np.pi / 180 + gmst
et_dec = et_location["latitude"] * np.pi / 180

In [ ]:
fits_files = {
    "et_only": "skymap_fits/ET_only/skymap.fits",
    "et_plus_ce": "skymap_fits/ET_CE/skymap.fits",
}
analyses = list(fits_files.keys())

labels = {
    "et_only": "ET only",
    "et_plus_ce": "ET + CE",
}

In [ ]:
from scipy.spatial.transform import Rotation as R
from astropy import constants
from astropy.coordinates import EarthLocation
from astropy import units as u
import numpy.lib.recfunctions as rfn

In [ ]:
det = EarthLocation.of_site("Virgo Observatory")
geocentre = np.zeros(3)
loc = rfn.structured_to_unstructured(det.value)
normal = loc - geocentre
D = np.dot(normal, loc)
distance = (500 * u.Mpc).to(u.meter).value

In [ ]:
print(f"Plane: {normal[0]}x + {normal[1]}y + {normal[2]}z = {D}")

In [ ]:
theta = np.linspace(0, 2 * np.pi, 100)

r = 1

ring = np.array(
    [
        r * np.sin(theta),
        r * np.cos(theta),
        r * np.zeros_like(theta),
    ]
)


normal = np.array([0, 0, 1])

# rot = R.from_euler("yz", [90 - et_location["latitude"], 360 - et_location["longitude"]], degrees=True)
rot = R.from_euler(
    "yz",
    [90 - et_location["latitude"], 180 - et_location["longitude"]],
    degrees=True,
)
# rot = R.from_euler("yz", [90 - 0.0, 180 - et_location["longitude"] - 5], degrees=True)
# rot = R.from_euler("yz", [np.pi / 2 - injected["dec"], injected["ra"] + np.pi], degrees=False)
rot = R.from_euler("yz", [np.pi / 2 - et_dec, et_ra], degrees=False)

ring_rot = rot.apply(ring.T).T

ax = plt.figure().add_subplot(projection="3d")

ax.plot(ring[0], ring[1], ring[2])

ax.plot(
    np.array([0, normal[0]]),
    np.array([0, normal[1]]),
    np.array([0, normal[2]]),
)

ax.plot(ring_rot[0], ring_rot[1], ring_rot[2])

normal_rot = rot.apply(normal).T
print(normal_rot.shape)

ax.plot(
    np.array([0, normal_rot[0]]),
    np.array([0, normal_rot[1]]),
    np.array([0, normal_rot[2]]),
)

# ring_rot[2] += 6378

# ring_rot /= np.sqrt(np.sum(ring_rot ** 2, axis=0))

ring_coord = SkyCoord(
    x=ring_rot[0],
    y=ring_rot[1],
    z=ring_rot[2],
    unit="m",
    representation_type="cartesian",
    # obstime=obs_time,
)
ring_coord.representation_type = "spherical"

normal_coord = SkyCoord(
    *normal_rot,
    unit="m",
    representation_type="cartesian",
    # obstime=obs_time,
)
normal_coord.representation_type = "spherical"

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

plt.show()

In [ ]:
idx = np.argsort(ring_coord.ra)
ring_ra, ring_dec = ring_coord.ra[idx], ring_coord.dec[idx]

In [ ]:
fig = plt.figure()
ax = plt.axes(
    projection="astro degrees mollweide",
    # center=SkyCoord("150deg -70deg"),
    # radius="20deg",
)
ax.grid()

ax.plot(
    ring_coord.ra,
    ring_coord.dec,
    transform=ax.get_transform("world"),
    color=thesis_colours.pillarbox,
)

center = SkyCoord(**injected, unit="rad")

ax.plot(
    center.ra.deg,
    center.dec.deg,
    transform=ax.get_transform("world"),
    marker=ligo.skymap.plot.reticle(),
    markersize=20,
    markeredgewidth=2,
    color="k",
)

ax.plot(
    180 - det.lon.deg,
    det.lat.deg,
    transform=ax.get_transform("world"),
    marker=ligo.skymap.plot.reticle(),
    markersize=20,
    markeredgewidth=2,
    color="k",
)

ax.plot(
    normal_coord.ra.deg,
    normal_coord.dec.deg,
    transform=ax.get_transform("world"),
    marker=ligo.skymap.plot.reticle(),
    markersize=20,
    markeredgewidth=2,
    color="green",
)

In [ ]:
fig = plt.figure()
ax = plt.axes(
    projection="astro degrees mollweide",
    # center=SkyCoord("150deg -70deg"),
    # radius="20deg",
)
ax.grid()

center = SkyCoord(**injected, unit="rad")

ax_inset = plt.axes(
    [0.9, 0.5, 0.3, 0.3],
    projection="astro zoom",
    radius=5 * u.deg,
    center=center,
)

for key in ["ra", "dec"]:
    ax_inset.coords[key].set_ticklabel_visible(False)
    ax_inset.coords[key].set_ticks_visible(False)

ax.mark_inset_axes(ax_inset)
ax.connect_inset_axes(ax_inset, "upper left")
ax.connect_inset_axes(ax_inset, "lower left")
ax_inset.scalebar((0.1, 0.1), 2 * u.deg).label()
ax_inset.compass(0.9, 0.1, 0.2)

# contours = 100 * np.array([1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.0)])
contours = [50, 90]
text = []
lw = 0.8

colours = ["C0", "C1"]

for analysis, colour in zip(analyses, colours):
    fits_file = fits_files[analysis]

    skymap, metadata = ligo.skymap.io.fits.read_sky_map(fits_file)
    nside = ah.npix_to_nside(len(skymap))
    deg2perpix = ah.nside_to_pixel_area(nside).to_value(u.deg**2)
    probperdeg2 = skymap / deg2perpix

    cls = 100 * ligo.skymap.postprocess.find_greedy_credible_levels(skymap)
    cs = ax.contour_hpx(
        (cls, "ICRS"),
        # nested=metadata['nest'],
        colors=colour,
        linewidths=lw,
        levels=contours,
        # smooth=0.9,
    )
    cs = ax_inset.contour_hpx(
        (cls, "ICRS"),
        # nested=metadata['nest'],
        colors=colour,
        linewidths=lw,
        levels=contours,
    )
    # ax.imshow_hpx(cls)
    pp = np.round(contours, 1).astype(float)
    ii = np.round(
        np.searchsorted(np.sort(cls), contours) * deg2perpix, 1
    ).astype(float)
    text_areas = []
    for i, p in zip(ii, pp):
        text_areas.append(f"{p}\%: {i}" + r"$\;\textrm{deg}^{2}$")
    text.append(f"{labels.get(analysis)}: {', '.join(text_areas)}")


ax_inset.plot(
    center.ra.deg,
    center.dec.deg,
    transform=ax_inset.get_transform("world"),
    marker=ligo.skymap.plot.reticle(),
    markersize=20,
    markeredgewidth=2,
    color="k",
)


handle_elements = [
    Line2D([0], [0], color="C0", label=text[0], lw=lw),
    Line2D([0], [0], color="C1", label=text[1], lw=lw),
]

# ax.text(1, -0.15, '\n'.join(text), transform=ax.transAxes, ha='right')
# ax.text(0, 1, event, transform=ax.transAxes, ha='left', fontsize=14)
ax.legend(
    handles=handle_elements,
    frameon=False,
    loc="center",
    bbox_to_anchor=(0.5, -0.2),
)

ax.plot(
    ring_ra,
    ring_dec,
    transform=ax.get_transform("world"),
    color=thesis_colours.pillarbox,
    ls="--",
)

ax.coords[1].set_ticks(spacing=30 * u.deg)
print(ax.coords[1].ticklabels.text)
ax.coords[1].set_ticklabel(exclude_overlapping=False)
print(ax.coords[1].ticklabels.text)


plt.show()
print(ax.coords[1].ticklabels.text)
# save_figure(fig, f"third_gen_skymap_comp", "figures", bbox_inches="tight")
plt.close("all")

In [ ]:
ax.coords

In [ ]:
ax.coords[1].ticklabels.text

In [ ]:
ax.coords[1].ticks.ticks_locs

In [ ]:
[k for k in dir(ax.coords[0]) if "tick" in k]